In [10]:
# R_t=simgoid(X_t@W_xr+H_t-1@W_hr+b_r)  X_t n*d  W_xr  d*h  W_hr h*h b_r h    n样本个数  d输入个数 h是隐藏层数
# Z_t=simgoid(X_t@W_xz+H_t-1@W_hz+b_z)  X_t n*d W_xz  d*h  W_hz h*h b_z h
# 候选隐状态 H_hat_t=tanh(X_t@W_xh+(R_t*H_t-1)*W_hh+b_h)
# H_t=Z_t * H_t-1+(1-Z_t) * H_hat_t
import sys
sys.path.append("D:\pysource\machine-study")

In [11]:
import torch
from torch import nn
from d2l import torch as d2l
from torch.nn import functional as F

batch_size, num_steps = 32, 35
train_iter, vocab = d2l.load_data_time_machine(batch_size, num_steps)

In [ ]:
class RNNModel():
    def __init__(self,num_hiddens,vocab_size):
        self.num_inputs=vocab_size
        self.num_outputs=vocab_size
        self.num_hiddens=num_hiddens
        self.vocab_size=vocab_size
        self.params=self.init_params()

    def init_params(self):

        def normal(size):
            return torch.randn(size=size)*0.01
        
        #重置门
        w_xr=normal((self.num_inputs,self.num_hiddens))
        w_hr=normal((self.num_hiddens,self.num_hiddens))
        b_r=torch.zeros(self.num_hiddens)
        #更新门
        w_xz=normal((self.num_inputs,self.num_hiddens))
        w_hz=normal((self.num_hiddens,self.num_hiddens))
        b_z=torch.zeros(self.num_hiddens)
        #候选隐状态
        w_xh=normal((self.num_inputs,self.num_hiddens))
        w_hh=normal((self.num_hiddens,self.num_hiddens))
        b_h=torch.zeros(self.num_hiddens)
        # 输出层
        w_hq=normal((self.num_hiddens,self.num_outputs))
        b_q=torch.zeros(self.num_outputs)

        params=[w_xr,w_hr,b_r,w_xz,w_hz,b_z,w_xh,w_hh,b_h,w_hq,b_q]
        for p in params:
            p.requires_grad_(True)
        return params
    
    def __call__(self,inputs,state):
        x_inputs=F.one_hot(inputs.T,self.vocab_size).to(torch.float32)
        h=state
        w_xr,w_hr,b_r,w_xz,w_hz,b_z,w_xh,w_hh,b_h,w_hq,b_q=self.params

        outputs=[]
        for x in x_inputs:
            R_t=torch.sigmoid((x @ w_xr)+(h @ w_hr)+b_r)
            Z_t=torch.sigmoid((x @ w_xz)+(h @ w_hz)+b_z)
            h_hat_t=torch.tanh((x@w_xh)+((R_t * h)@w_hh)+b_h)
            h=Z_t * h + (1-Z_t) * h_hat_t
            y=h @ w_hq +b_q
            outputs.append(y)
        return torch.cat(outputs,dim=0),h
    
    def begin_state(self,batch_size):
        return torch.zeros((batch_size,self.num_hiddens))
        

In [ ]:
# 训练
# 1. 定义损失函数
# 2  定义梯度更新方式
# 3  预热数据
# 4  训练
from torch import nn


def pred_ch8(prefix,pred_num,net,vocab):
    state=net.begin_state(1)
    outputs=[vocab[prefix[0]]]
    get_input=lambda: torch.tensor([outputs[-1]]).reshape(1,1)
    for y in prefix[1:]: #预热
        _,state=net(get_input(),state)
        outputs.append(vocab[y])
    for _ in range(pred_num): #预测
        y,state=net(get_input(),state)
        outputs.append(int(y.argmax(dim=1).reshape(1)))
    return ''.join([vocab.idx_to_token[i] for i in outputs])

# 梯度裁剪  g<-min(1,theta/||g||)g
def grad_clipping(net,theta):
    params=net.params
    # for p in params:
    #     print(p.dtype)
    norm=torch.sqrt(sum(torch.sum((p.grad**2)) for p in params))
    if norm>theta:
        for p in params:
            p.grad[:]*= theta/norm

def sgd(params,lr):
    with torch.no_grad():
        for p in params:
            p-=lr*p.grad
            p.grad.zero_()

def train_epoch_ch8(net,loss,updater,train_iter,lr):
    state=None
    for X,Y in train_iter:
        if state==None:
            state=net.begin_state(X.shape[0])
        else:
            state.detach_()  #分离state
        y=Y.T.reshape(-1)
        y_hat,state=net(X,state)
        l=loss(y_hat,y.long()).mean()
        l.backward()
        grad_clipping(net,1)
        updater(net.params,lr)

def train_ch8(net,vocab,lr,epoch,train_iter):
    loss=nn.CrossEntropyLoss()
    updater=sgd
    predict=lambda prefix: pred_ch8(prefix,50,net,vocab)
    for _ in range(epoch):
        train_epoch_ch8(net,loss,updater,train_iter,lr)
    print(predict("time traveller"))
    print(predict("traveller"))


epoch=500
lr=1
num_hiddens=256
net=RNNModel(num_hiddens,len(vocab))
train_ch8(net,vocab,lr,epoch,train_iter)

torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32


TypeError: unsupported operand type(s) for ** or pow(): 'NoneType' and 'int'